In [1]:
import os
import torch
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm.notebook import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from functools import partial
import optuna
import gc
from typing import Literal
import torch.nn.functional as F
from torch.utils.data import Subset, Dataset

# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN


# Set the random seed
set_seed()


In [ ]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [3]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))

    os.makedirs(submission_folder, exist_ok=True)

    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")

    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({"id": test_graph_ids, "pred": predictions})

    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [4]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color="blue")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss per Epoch")

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color="green")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training Accuracy per Epoch")

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [5]:
class IndexedSubset(Dataset):
    def __init__(self, subset: Subset):
        self.subset = subset

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, i):
        data = self.subset[i]
        data.idx = torch.tensor(i, dtype=torch.long)  # type: ignore
        return data

In [6]:
class NCODLoss(nn.Module):
    past_embeddings: torch.Tensor
    centroids: torch.Tensor

    def __init__(
        self,
        dataset: Dataset,
        embedding_dimensions: int = 300,
        total_epochs: int = 150,
        lambda_consistency: float = 1.0,
        device: torch.device | None = None,
    ):
        """
        Args
        ----
        dataset : iterable whose elements expose an integer label in `elem.y`
        embedding_dimensions : size of the feature vectors
        total_epochs : number of training epochs (used for centroid update schedule)
        lambda_consistency : weight for the MSE consistency term
        device : cuda / cpu device.  If None, picks CUDA if available.
        """
        super().__init__()

        self.device = device or torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        self.embedding_dimensions = embedding_dimensions
        self.total_epochs = total_epochs
        self.lambda_consistency = lambda_consistency

        labels = [int(elem.y) for elem in dataset]
        self.num_elements = len(labels)
        self.num_classes = max(labels) + 1  # robust to gaps (e.g. labels {1,3})
        # self.register_buffer("bins", torch.empty(self.num_classes, 0, dtype=torch.long))

        # Convert bins to a list-of-lists for easy appends, then to tensors
        tmp_bins: list[list[int]] = [[] for _ in range(self.num_classes)]
        for idx, lab in enumerate(labels):
            tmp_bins[lab].append(idx)
        self.bins = [
            torch.as_tensor(b, dtype=torch.long, device=self.device) for b in tmp_bins
        ]

        # Confidence parameter per sample (trainable!)
        self.u = nn.Parameter(torch.empty(self.num_elements, 1, device=self.device))
        nn.init.normal_(self.u, mean=1e-8, std=1e-9)

        # Running memory of embeddings
        self.register_buffer(
            "past_embeddings",
            torch.rand(
                self.num_elements, self.embedding_dimensions, device=self.device
            ),
        )
        # Class centroids
        self.register_buffer(
            "centroids",
            torch.rand(self.num_classes, self.embedding_dimensions, device=self.device),
        )

    def forward(
        self,
        *,
        logits: torch.Tensor,  # (B, C)
        indexes: torch.Tensor,  # (B,) – dataset indices of the current batch
        embeddings: torch.Tensor,  # (B, D)
        targets: torch.Tensor,  # (B,)
        epoch: int,
    ) -> torch.Tensor:
        eps = 1e-6

        # Keep an L2-normalised copy of current embeddings
        embeddings = F.normalize(embeddings, dim=1)
        self.past_embeddings[indexes] = embeddings.detach()

        # ---------------- Centroid update ------------------------------------
        if epoch == 0:
            with torch.no_grad():
                for c, idxs in enumerate(self.bins):
                    if idxs.numel():
                        self.centroids[c] = self.past_embeddings[idxs].mean(0)
        else:
            # Shrink the subset of samples that contribute to the centroid
            percent = int(max(1, min(100, 50 + 50 * (1 - epoch / self.total_epochs))))
            for c, idxs in enumerate(self.bins):
                if idxs.numel() == 0:
                    continue
                # bottom-k u’s  (small u  ⇒ low confidence ⇒ smaller weight)
                k = max(1, idxs.numel() * percent // 100)
                u_batch = self.u[idxs].squeeze(1)
                keep = torch.topk(u_batch, k, largest=False).indices  # (k,)
                selected = idxs[keep]  # (k,)
                self.centroids[c] = self.past_embeddings[selected].mean(0)

        centroids = F.normalize(self.centroids, dim=1)  # (C, D)

        # ---------------- Probability shaping --------------------------------
        soft_labels = F.softmax(embeddings @ centroids.T, dim=1)  # (B, C)
        probs = F.softmax(logits, dim=1)  # (B, C)
        u_vals = torch.sigmoid(self.u[indexes]).squeeze(1)  # (B,)

        adjusted = (probs + u_vals[:, None] * soft_labels).clamp(min=eps)
        adjusted = adjusted / adjusted.sum(1, keepdim=True)

        # ---------------- Loss terms -----------------------------------------
        hard_ce = (
            (1.0 - u_vals) * F.cross_entropy(logits, targets, reduction="none")
        ).mean()
        soft_ce = -(soft_labels * torch.log(adjusted)).sum(1).mean()
        consistency = F.mse_loss(adjusted, soft_labels)

        return hard_ce + soft_ce + self.lambda_consistency * consistency


In [7]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (
            1
            - torch.nn.functional.one_hot(targets, num_classes=logits.size(1))
            .float()
            .sum(dim=1)
        )
        return (losses * weights).mean()

In [8]:
class SCELoss(torch.nn.Module):
    def __init__(self, num_classes: int = 6, alpha: float = 0.1, beta: float = 1.0):
        super().__init__()
        self.alpha, self.beta = alpha, beta
        self.num_classes = num_classes

    def forward(self, logits, targets):
        # CCE
        ce = F.cross_entropy(logits, targets, reduction="none")

        # RCE
        pred = F.softmax(logits, dim=1).clamp(min=1e-6, max=1 - 1e-6)
        one_hot = F.one_hot(targets, self.num_classes).float()
        rce = -(1 - one_hot) * torch.log(1 - pred)
        rce = rce.sum(dim=1)
        return (self.alpha * ce + self.beta * rce).mean()

In [9]:
def train(
    data_loader: DataLoader,
    model: GNN,
    optimizer_theta: torch.optim.Optimizer,
    optimizer_u: torch.optim.Optimizer | None,
    criterion: nn.Module,
    device: torch.device,
    checkpoint_path: str,
    current_epoch: int,
    save_checkpoints: bool = True,
):
    model.train()

    total_loss = total_conf = total_entropy = 0.0
    correct = num_samples = 0

    for batch in data_loader:
        batch = batch.to(device)

        node_emb = model.gnn_node(batch)
        graph_emb = model.pool(node_emb, batch.batch)
        logits = model.graph_pred_linear(graph_emb)

        if isinstance(criterion, NCODLoss):
            loss = criterion(
                logits=logits,
                indexes=batch.idx.to(device),
                embeddings=graph_emb,
                targets=batch.y.to(device),
                epoch=current_epoch,
            )
        else:
            loss = criterion(logits, batch.y)

        optimizer_theta.zero_grad(set_to_none=True)
        if optimizer_u is not None:
            optimizer_u.zero_grad(set_to_none=True)
        loss.backward()
        optimizer_theta.step()
        if optimizer_u is not None:
            optimizer_u.step()

        with torch.no_grad():
            probs = F.softmax(logits, dim=1)
            batch_size = batch.y.size(0)

            total_loss += loss.item() * batch_size
            pred = probs.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
            num_samples += batch_size

            total_conf += probs.max(dim=1).values.sum().item()
            total_entropy += (
                (-torch.sum(probs * torch.log(probs + 1e-10), 1)).sum().item()
            )

    if save_checkpoints:
        ckpt = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), ckpt)
        print(f"[checkpoint] saved: {ckpt}")

    loss = total_loss / num_samples
    confidence = total_conf / num_samples
    entropy = total_entropy / num_samples
    accuracy = correct / num_samples
    return loss, confidence, accuracy, entropy


In [10]:
def evaluate(
    data_loader: DataLoader,
    model: GNN,
    criterion: nn.CrossEntropyLoss,
    device: torch.device,
) -> tuple[float, float, float, float]:
    """
    Returns
    -------
    avg_loss, avg_confidence, accuracy, avg_entropy
    """
    model.eval()

    total_loss = total_conf = total_entropy = 0.0
    correct = num_samples = 0

    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)

            node_emb = model.gnn_node(batch)
            graph_emb = model.pool(node_emb, batch.batch)
            logits = model.graph_pred_linear(graph_emb)
            probs = F.softmax(logits, dim=1)

            loss = criterion(logits, batch.y)

            batch_size = batch.y.size(0)
            total_loss += loss.item() * batch_size

            total_conf += probs.max(dim=1).values.sum().item()
            total_entropy += (
                (-torch.sum(probs * torch.log(probs + 1e-10), dim=1)).sum().item()
            )

            pred = probs.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
            num_samples += batch_size

    loss = total_loss / num_samples
    confidence = total_conf / num_samples
    entropy = total_entropy / num_samples
    accuracy = correct / num_samples

    return loss, confidence, accuracy, entropy


In [11]:
def objective(
    trial,
    train_loader: DataLoader,
    val_loader: DataLoader,
    num_checkpoints: int,
    checkpoints_dir: str,
    run_name: str,
    best_model_path: str,
    logs_dir: str,
    *,
    score_type: Literal["loss", "accuracy", "entropy", "confidence","composite"] = "composite",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
) -> float:
    try:
        logging.info("#" * 80)
        # Hyperparameter search space
        logging.info("Start case study with parameters:")
        gnn_type = trial.suggest_categorical(
            "gnn_type", ["gin", "gin-virtual", "gcn", "gcn-virtual"]
        )
        loss_type = trial.suggest_categorical(
            "loss_type", ["ce"] + ["ncod"] * 5 + ["noisy_ce"] * 5 + ["sce"]
        )
        graph_pooling = trial.suggest_categorical(
            "graph_pooling",
            ["sum"] * 2 + ["mean"] * 4 + ["max"] * 2 + ["attention"] * 2 + ["set2set"],
        )
        gnn_type = "gcn"
        graph_pooling ="mean"
        loss_type = "nosy_ce"
        drop_ratio = trial.suggest_float("dropout", 0.0, 0.7)
        num_layers = trial.suggest_int("num_layers", 3, 12)
        embedding_dim = trial.suggest_categorical("embedding_dim", [64, 128, 300,600])
        num_epochs = 20

        print(
            f"{gnn_type=}\n{loss_type=}\n{graph_pooling=}\n{drop_ratio=}\n{num_layers=}\n{embedding_dim=}\n{num_epochs=}"
        )
        logging.info(
            f"{gnn_type=} {loss_type=} {graph_pooling=} {drop_ratio=} {num_layers=} {embedding_dim=} {num_epochs=}"
        )

        # Initialize model
        model = GNN(
            gnn_type="gin" if "gin" in gnn_type else "gcn",
            num_class=6,
            num_layer=num_layers,
            emb_dim=2 * embedding_dim
            if graph_pooling == "set2set" and loss_type == "ncod"
            else embedding_dim,
            drop_ratio=drop_ratio,
            virtual_node="virtual" in gnn_type,
            graph_pooling=graph_pooling,
        ).to(device)

        optimizer_theta = torch.optim.Adam(
            model.parameters(), lr=3e-4, weight_decay=1e-5
        )
        optimizer_u = None
        if loss_type == "ce":
            train_criterion = nn.CrossEntropyLoss()
        elif loss_type == "ncod":
            train_criterion = NCODLoss(
                train_loader.dataset,
                embedding_dimensions=embedding_dim,
                total_epochs=num_epochs,
                device=device,
            )
            optimizer_u = torch.optim.SGD(train_criterion.parameters(), lr=1e-3)
        elif loss_type == "sce":
            train_criterion = SCELoss()
        else:
            train_criterion = NoisyCrossEntropyLoss(0.2)

        val_criterion = nn.CrossEntropyLoss()

        # Checkpoint logic
        checkpoint_epochs = [
            int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)
        ]

        train_losses, train_confs, train_accs, train_entropies, train_scores = (
            [],
            [],
            [],
            [],
            [],
        )
        best_val_score = -float("inf")
        val_losses, val_confs, val_accs, val_entropies, val_scores = [], [], [], [], []
        
        progress_bar = tqdm(range(num_epochs), desc="Training...", leave=False)
        for epoch in progress_bar:
            train_loss, train_conf, train_acc, train_entropy = train(
                train_loader,
                model,
                optimizer_theta,
                optimizer_u,
                train_criterion,
                device,
                save_checkpoints=(epoch + 1 in checkpoint_epochs),
                checkpoint_path=os.path.join(checkpoints_dir, f"model_{run_name}"),
                current_epoch=epoch,
            )

            val_loss, val_conf, val_acc, val_entropy = evaluate(
                val_loader,
                model,
                val_criterion,
                device,
            )

            train_losses.append(train_loss)
            train_accs.append(train_acc)
            train_confs.append(train_conf)
            train_entropies.append(train_entropy)

            val_losses.append(val_loss)
            val_accs.append(val_acc)
            val_confs.append(val_conf)
            val_entropies.append(val_entropy)

            if score_type == "loss":
                train_score = train_loss
                val_score = -val_loss
            elif score_type == "entropy":
                train_score = train_entropy
                val_score = -val_entropy
            elif score_type == "confidence":
                train_score = train_conf
                val_score = val_conf

            else:
                train_score = train_acc
                val_score = val_acc

            train_scores.append(train_score)
            val_scores.append(val_score)
            trial.report(val_score, step=epoch)

            if val_score > best_val_score:
                best_val_score = val_score
                torch.save(model.state_dict(), best_model_path)
                logging.info(f"[{run_name}] Best model updated at {best_model_path}")
            progress_bar.set_postfix_str(
                f"Train Score: {train_score:.4f}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, Confidence: {train_conf:.4f}, Entropy: {train_entropy:.4f}| "
                f"Val Score: {val_score:.4f}, Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, Confidence: {val_conf:.4f}, Entropy: {val_entropy:.4f}"
            )
            logging.info(
                f"Epoch {epoch}/{num_epochs}| "
                f"Train Score: {train_score:.4f}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, Confidence: {train_conf:.4f}, Entropy: {train_entropy:.4f}| "
                f"Val Score: {val_score:.4f}, Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, Confidence: {val_conf:.4f}, Entropy: {val_entropy:.4f}"
            )
            if trial.should_prune():
                logging.warning(f"Trial was pruned at epoch {epoch}")
                raise optuna.exceptions.TrialPruned()
           

        # Plot training curves
        plot_training_progress(
            train_losses, train_scores, os.path.join(logs_dir, "train_plots")
        )
        plot_training_progress(
            val_losses, val_scores, os.path.join(logs_dir, "val_plots")
        )

        return best_val_score
    except optuna.exceptions.TrialPruned as e:
        raise e

    except Exception as e:
        print("Unhandled exception ", e)
        return -float("inf")

    finally:
        logging.info(f"Case study end, {best_val_score}")
        logging.info("#" * 80)
        logging.info("\n")
        progress_bar.close()

In [12]:
from torch.utils.data import WeightedRandomSampler
import numpy as np
from optuna.pruners import MedianPruner


def case_study(
    dataset_name: Literal["A", "B", "C", "D"],
    n_trials: int = 30,
    num_checkpoints: int = 5,
    default_batch_size: int = 64,
    score_type: Literal["loss", "accuracy", "entropy", "confidence","composite"] = "accuracy",
    full_dataset=None,
):
    script_root = os.getcwd()
    train_path = f"./datasets/{dataset_name}/train.json.gz"
    run_name = dataset_name

    logs_dir = os.path.join(script_root, "logs", run_name)
    os.makedirs(logs_dir, exist_ok=True)
    logging.basicConfig(
        filename=os.path.join(logs_dir, "training.log"),
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        filemode="w",
    )

    checkpoints_dir = os.path.join(script_root, "checkpoints", run_name)
    best_model_path = os.path.join(checkpoints_dir, f"model_{run_name}_best.pth")
    summary_csv_path = os.path.join(logs_dir, f"optuna_summary_{dataset_name}.csv")
    os.makedirs(checkpoints_dir, exist_ok=True)

    if full_dataset is None:
        full_dataset = GraphDataset(train_path, transform=add_zeros)

    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(
        full_dataset, [train_size, val_size], generator=generator
    )
    train_dataset = IndexedSubset(train_dataset)
    val_dataset = IndexedSubset(val_dataset)

    # ---- WeightedRandomSampler for class balancing ----
    labels = [int(full_dataset[i].y[0]) for i in train_dataset.subset.indices]
    class_counts = np.bincount(labels)
    class_weights = 1.0 / class_counts
    sample_weights = [class_weights[label] for label in labels]
    sampler = WeightedRandomSampler(
        sample_weights, num_samples=len(sample_weights), replacement=True
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=default_batch_size,
        sampler=sampler,  # replaces shuffle=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=default_batch_size,
        shuffle=False,
    )

    print(f"Starting Optuna optimization for dataset {dataset_name}")

    study = optuna.create_study(
        study_name=run_name,
        direction="maximize",
        pruner=MedianPruner(n_warmup_steps=10),
        sampler=optuna.samplers.TPESampler(n_startup_trials=0),
    )

    obj = partial(
        objective,
        train_loader=train_loader,
        val_loader=val_loader,
        num_checkpoints=num_checkpoints,
        checkpoints_dir=checkpoints_dir,
        run_name=run_name,
        best_model_path=best_model_path,
        logs_dir=logs_dir,
        score_type=score_type,
    )
    study.optimize(obj, n_trials=n_trials, show_progress_bar=True)

    all_trials = []
    for trial in study.trials:
        if trial.state == optuna.trial.TrialState.COMPLETE:
            row = {score_type: trial.value}
            row.update(trial.params)
            all_trials.append(row)

    results_df = pd.DataFrame(all_trials).sort_values(score_type, ascending=False)
    results_df.to_csv(summary_csv_path, index=False)

    print(f"\nAll trials saved to: {summary_csv_path}")
    print(f"\nBest result for dataset {dataset_name}:")
    display(results_df)
    print(f"\nBest Params for {dataset_name}:")
    for k, v in study.best_params.items():
        print(f"  {k}: {v}")

    del train_loader, val_loader, full_dataset, train_dataset, val_dataset
    gc.collect()


In [13]:
case_study(
    "A", n_trials=60, num_checkpoints=5, default_batch_size=64, score_type="accuracy"
)

[I 2025-05-29 23:13:40,736] A new study created in memory with name: A


Starting Optuna optimization for dataset A


  0%|          | 0/60 [00:00<?, ?it/s]

gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.36523063702616004
num_layers=12
embedding_dim=128
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

...

[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_4.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_8.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_12.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_16.pth
[I 2025-05-30 04:36:08,225] Trial 58 pruned. 
gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.37682350934254916
num_layers=6
embedding_dim=128
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_4.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_8.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_12.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_16.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/A/model_A_epoch_20.pth
[I 2025-05-30 04:38:54,585] Trial 59 finished with value: 0.5270390070921985 and parameters: {'gnn_type': 'gin-virtual', 'loss_type': 'ncod', 'graph_pooling': 'mean', 'dropout': 0.37682350934254916, 'num_layers': 6, 'embedding_dim': 128}. Best is trial 35 with value: 0.5651595744680851.

All trials saved to: /home/haislich/Documents/noisy_labels/hackaton/logs/A/optuna_summary_A.csv

Best result for dataset A:


,accuracy,gnn_type,loss_type,graph_pooling,dropout,num_layers,embedding_dim
21,0.565160,gin-virtual,noisy_ce,mean,0.407236,6,300
13,0.562500,gin-virtual,ncod,sum,0.282099,6,600
28,0.562500,gcn-virtual,sce,sum,0.260906,12,300
7,0.554078,gin-virtual,ncod,sum,0.237048,6,300
12,0.553635,gin-virtual,ncod,max,0.295029,8,128
6,0.552748,gin-virtual,noisy_ce,sum,0.267366,7,300
3,0.550532,gcn-virtual,sce,mean,0.200908,12,300
9,0.550532,gin-virtual,noisy_ce,sum,0.266574,4,300
19,0.548316,gin-virtual,noisy_ce,mean,0.281055,7,300
14,0.546099,gin-virtual,ncod,mean,0.314500,5,600



Best Params for A:
  gnn_type: gin-virtual
  loss_type: noisy_ce
  graph_pooling: mean
  dropout: 0.4072362823036999
  num_layers: 6
  embedding_dim: 300


In [14]:
case_study(
    "B", n_trials=60, num_checkpoints=5, default_batch_size=64, score_type="accuracy"
)

[I 2025-05-30 04:38:56,034] A new study created in memory with name: B


Starting Optuna optimization for dataset B


  0%|          | 0/60 [00:00<?, ?it/s]

gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.1301692261498399
num_layers=9
embedding_dim=128
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

...

[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_4.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_8.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_12.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_16.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_20.pth
[I 2025-05-30 06:17:53,750] Trial 58 finished with value: 0.5133928571428571 and parameters: {'gnn_type': 'gcn', 'loss_type': 'ncod', 'graph_pooling': 'attention', 'dropout': 0.061584013448205244, 'num_layers': 4, 'embedding_dim': 64}. Best is trial 13 with value: 0.5241071428571429.
gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.04757262891930081
num_layers=4
embedding_dim=64
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_4.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_8.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_12.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_16.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/B/model_B_epoch_20.pth
[I 2025-05-30 06:18:28,805] Trial 59 finished with value: 0.49642857142857144 and parameters: {'gnn_type': 'gcn', 'loss_type': 'noisy_ce', 'graph_pooling': 'mean', 'dropout': 0.04757262891930081, 'num_layers': 4, 'embedding_dim': 64}. Best is trial 13 with value: 0.5241071428571429.

All trials saved to: /home/haislich/Documents/noisy_labels/hackaton/logs/B/optuna_summary_B.csv

Best result for dataset B:


,accuracy,gnn_type,loss_type,graph_pooling,dropout,num_layers,embedding_dim
11,0.524107,gcn,ncod,sum,0.206049,3,64
19,0.518750,gin,ncod,mean,0.252029,3,300
37,0.514286,gcn,ncod,attention,0.201135,3,600
47,0.513393,gcn,ncod,attention,0.061584,4,64
9,0.509821,gcn,ncod,mean,0.331577,5,64
42,0.500000,gcn,noisy_ce,sum,0.280014,3,600
30,0.497321,gin-virtual,ncod,attention,0.256647,3,128
34,0.496429,gcn,ncod,sum,0.296272,3,600
48,0.496429,gcn,noisy_ce,mean,0.047573,4,64
35,0.496429,gcn,noisy_ce,max,0.292967,3,600



Best Params for B:
  gnn_type: gcn
  loss_type: ncod
  graph_pooling: sum
  dropout: 0.20604927082840657
  num_layers: 3
  embedding_dim: 64


In [15]:
case_study(
    "C", n_trials=60, num_checkpoints=5, default_batch_size=64, score_type="accuracy"
)

[I 2025-05-30 06:18:30,651] A new study created in memory with name: C


Starting Optuna optimization for dataset C


  0%|          | 0/60 [00:00<?, ?it/s]

gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.4846392046162389
num_layers=5
embedding_dim=300
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

...

[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/C/model_C_epoch_4.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/C/model_C_epoch_8.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/C/model_C_epoch_12.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/C/model_C_epoch_16.pth
[I 2025-05-30 09:39:36,170] Trial 58 pruned. 
gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.33152247181166733
num_layers=12
embedding_dim=600
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

Unhandled exception  CUDA out of memory. Tried to allocate 636.00 MiB. GPU 0 has a total capacity of 11.60 GiB of which 322.12 MiB is free. Process 6163 has 39.59 MiB memory in use. Including non-PyTorch memory, this process has 10.83 GiB memory in use. Of the allocated memory 8.91 GiB is allocated by PyTorch, and 1.70 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
[I 2025-05-30 09:39:36,391] Trial 59 finished with value: -inf and parameters: {'gnn_type': 'gcn-virtual', 'loss_type': 'noisy_ce', 'graph_pooling': 'max', 'dropout': 0.33152247181166733, 'num_layers': 12, 'embedding_dim': 600}. Best is trial 41 with value: 0.712890625.

All trials saved to: /home/haislich/Documents/noisy_labels/hackaton/logs/C/optuna_summary_C.csv

Best result for dataset

,accuracy,gnn_type,loss_type,graph_pooling,dropout,num_layers,embedding_dim
18,0.712891,gcn-virtual,sce,max,0.226569,10,300
14,0.699219,gcn,ncod,mean,0.463611,4,300
4,0.694010,gcn-virtual,ncod,set2set,0.348624,11,300
21,0.692708,gcn-virtual,noisy_ce,mean,0.311626,12,300
20,0.688802,gcn-virtual,noisy_ce,sum,0.271676,11,300
5,0.679688,gcn-virtual,sce,set2set,0.333198,11,300
19,0.673828,gcn-virtual,noisy_ce,sum,0.267540,11,300
24,0.670573,gcn-virtual,noisy_ce,mean,0.316919,11,300
0,0.663411,gcn,noisy_ce,attention,0.484639,5,300
11,0.660156,gcn-virtual,sce,mean,0.252603,10,300



Best Params for C:
  gnn_type: gcn-virtual
  loss_type: sce
  graph_pooling: max
  dropout: 0.22656907931569426
  num_layers: 10
  embedding_dim: 300


In [16]:
case_study(
    "D", n_trials=60, num_checkpoints=5, default_batch_size=64, score_type="accuracy"
)

[I 2025-05-30 09:39:39,024] A new study created in memory with name: D


Starting Optuna optimization for dataset D


  0%|          | 0/60 [00:00<?, ?it/s]

gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.11138062922411074
num_layers=12
embedding_dim=64
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

...

[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/D/model_D_epoch_4.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/D/model_D_epoch_8.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/D/model_D_epoch_12.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/D/model_D_epoch_16.pth
[checkpoint] saved: /home/haislich/Documents/noisy_labels/hackaton/checkpoints/D/model_D_epoch_20.pth
[I 2025-05-30 13:14:29,829] Trial 58 finished with value: 0.7028210116731517 and parameters: {'gnn_type': 'gin-virtual', 'loss_type': 'noisy_ce', 'graph_pooling': 'attention', 'dropout': 0.05487534801392718, 'num_layers': 4, 'embedding_dim': 600}. Best is trial 58 with value: 0.7028210116731517.
gnn_type='gcn'
loss_type='nosy_ce'
graph_pooling='mean'
drop_ratio=0.05776506675573531
num_layers=11
embedding_dim=600
num_epochs=20


Training...:   0%|          | 0/20 [00:00<?, ?it/s]

Unhandled exception  CUDA out of memory. Tried to allocate 654.00 MiB. GPU 0 has a total capacity of 11.60 GiB of which 342.25 MiB is free. Process 6163 has 39.59 MiB memory in use. Including non-PyTorch memory, this process has 10.67 GiB memory in use. Process 58091 has 135.84 MiB memory in use. Of the allocated memory 9.07 GiB is allocated by PyTorch, and 1.37 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
[I 2025-05-30 13:14:30,046] Trial 59 finished with value: -inf and parameters: {'gnn_type': 'gin-virtual', 'loss_type': 'ncod', 'graph_pooling': 'mean', 'dropout': 0.05776506675573531, 'num_layers': 11, 'embedding_dim': 600}. Best is trial 58 with value: 0.7028210116731517.

All trials saved to: /home/haislich/Documents/noisy_labels/hackaton/logs

,accuracy,gnn_type,loss_type,graph_pooling,dropout,num_layers,embedding_dim
20,0.702821,gin-virtual,noisy_ce,attention,0.054875,4,600
10,0.689689,gin-virtual,noisy_ce,mean,0.499422,10,300
11,0.685798,gin-virtual,ncod,attention,0.519216,10,300
5,0.683852,gcn,ncod,mean,0.298479,7,600
15,0.681907,gin-virtual,ncod,mean,0.358423,10,300
12,0.676556,gin-virtual,ncod,sum,0.513133,11,300
19,0.676556,gcn-virtual,noisy_ce,sum,0.150189,6,64
7,0.672665,gcn-virtual,noisy_ce,mean,0.228287,9,64
14,0.664883,gcn,noisy_ce,attention,0.627217,5,600
4,0.661479,gcn-virtual,ce,mean,0.291027,7,64



Best Params for D:
  gnn_type: gin-virtual
  loss_type: noisy_ce
  graph_pooling: attention
  dropout: 0.05487534801392718
  num_layers: 4
  embedding_dim: 600
